# SDB2 - Projeto de análise de dados de sinistros (PRF)

## Imports das bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore') # somente para ignorar os warnings
from tqdm import tqdm_notebook as tqdm # barra de progresso
import folium # Para usar o mapa
import folium.plugins as plugins # Para usar o cluster
import branca

## Import dos Dataframes

In [ ]:
df = pd.read_csv('../silver/database/sinistros_tratado.csv', sep=';', encoding='latin1', quotechar='"', low_memory=False)

## Tratamento para exibição no Mapa

### Filtrar apenas para a causa principal

In [ ]:
df = df[df['CausaPrincipal'] == 'Sim']

### Agrupar por sinistro

In [ ]:
df = df.groupby('SinistroID').agg({
    'Causa': 'first',
    'TipoDoSinistro': 'first',
    'DataHora': 'first',
    'UF': 'first',
    'Localidade': 'first',
    'Regiao': 'first',
    'Municipio': 'first',
    'Latitude': 'first',
    'Longitude': 'first',
    'Ilesos': 'sum',
    'FeridosLeves': 'sum',
    'FeridosGraves': 'sum',
    'Feridos': 'sum',
    'Mortos': 'sum',
    'UPS': 'max',
}).reset_index()

### Adiciona Gravidade para cada sinistro agrupado

In [ ]:
# Variáveis para definir a gravidade do sinistro
mortos = pd.to_numeric(df.get('Mortos'), errors='coerce').fillna(0)
feridos = pd.to_numeric(df.get('Feridos'), errors='coerce').fillna(0)

# Define condições para a coluna Gravidade
cond_morto = mortos > 0
cond_ferido = feridos.notna() & (feridos > 0)
cond_sem_vitima = feridos.notna() & (feridos == 0)

choices = ['Com óbito', 'Com ferido', 'Sem ferido']
conds = [cond_morto, cond_ferido, cond_sem_vitima]

df['Gravidade'] = pd.Series(np.select(conds, choices, default='Não informado'), index=df.index).astype('string')

### Exibe informações do Dataset

In [ ]:
df.info()

### Limpa dados geográficos

In [ ]:
df = df.dropna(subset=['Latitude', 'Longitude'])
df = df[(df['Latitude'] != 0) & (df['Longitude'] != 0)]

## Otimização do Dataset para Mapa

### Aplicação de filtros opcionais

In [ ]:
MAX_PONTOS = None  # Limite máximo de pontos no mapa
FILTRO_UF = None   # Ex: 'DF', 'GO' para filtrar por UFs específicas
FILTRO_ANO = None  # Ex: 2024 para filtrar apenas um ano
FILTRO_REGIAO = None  # Ex: 'Centro-Oeste', 'Norte' para filtrar por regiões específicas

In [ ]:
df_mapa = df.copy()

# Filtrar por pontos com amostragem estratificada por gravidade para manter representatividade se especificado
if MAX_PONTOS and len(df_mapa) > MAX_PONTOS:
    # Calcular proporções de cada categoria de gravidade
    prop_gravidade = df_mapa['Gravidade'].value_counts(normalize=True)
    
    # Calcular quantos pontos de cada categoria manter
    amostras_por_gravidade = {}
    for gravidade, prop in prop_gravidade.items():
        amostras_por_gravidade[gravidade] = int(MAX_PONTOS * prop)
    
    # Fazer amostragem estratificada
    df_amostrados = []
    for gravidade, n_amostras in amostras_por_gravidade.items():
        df_categoria = df_mapa[df_mapa['Gravidade'] == gravidade]
        if len(df_categoria) <= n_amostras:
            df_amostrados.append(df_categoria)
        else:
            df_amostrados.append(df_categoria.sample(n=n_amostras, random_state=42))
    
    df_mapa = pd.concat(df_amostrados, ignore_index=True)

# Filtrar por ano se especificado
if FILTRO_ANO:
    df_mapa = df_mapa[df_mapa['Data'].dt.year == FILTRO_ANO]

# Filtrar por UFs se especificado
if FILTRO_UF:
    df_mapa = df_mapa[df_mapa['UF'].isin([FILTRO_UF])]

# Filtrar por regiões se especificado
if FILTRO_REGIAO:
    df_mapa = df_mapa[df_mapa['Regiao'].isin([FILTRO_REGIAO])]

# Construção do Mapa de Dados Otimizado

### Criação do mapa rodoviário centralizado no Brasil

In [ ]:
m = folium.Map(
    location = [-14.65891498638056, -51.93789489425806], # Centro do Brasil
    # location = [-16.766535559581794, -53.655875080657665], # Centro do Centro-Oeste
    zoom_start = 4.5, # 4.5 para o Brasil, 6 para o Centro-Oeste
    tiles = 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png',
    attr='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors',
    prefer_canvas=True
)

### Criação dos clusters por gravidade do sinistro

In [ ]:
marker_cluster_sem_ferido = plugins.MarkerCluster(
    name='Sem ferido',
    options={'maxClusterRadius': 50, 'disableClusteringAtZoom': 10}
).add_to(m)
marker_cluster_com_ferido = plugins.MarkerCluster(
    name='Com ferido',
    options={'maxClusterRadius': 50, 'disableClusteringAtZoom': 10}
).add_to(m)
marker_cluster_com_obito = plugins.MarkerCluster(
    name='Com óbito',
    options={'maxClusterRadius': 50, 'disableClusteringAtZoom': 10}
).add_to(m)

### Criação do popup com os dados de cada sinistro

In [ ]:
# Função HTML para popup
def fancy_html(row):
    i = row

    SinistroID = df_mapa['SinistroID'].iloc[i]
    Regiao = df_mapa['Regiao'].iloc[i]
    UF = df_mapa['Localidade'].iloc[i]
    Municipio = df_mapa['Municipio'].iloc[i].lower().title()
    TipoDoSinistro = df_mapa['TipoDoSinistro'].iloc[i]
    Causa = df_mapa['Causa'].iloc[i]
    Gravidade = df_mapa['Gravidade'].iloc[i]
    UPS = df_mapa['UPS'].iloc[i]

    html = f"""
    <div style="font-family: Arial, sans-serif; width: 320px; padding: 10px;">
        <h4 style="margin: 0 0 10px 0; color: #2A799C; border-bottom: 2px solid #2A799C; padding-bottom: 5px;">
            Sinistro ID: {SinistroID}
        </h4>
        <table style="width: 100%; border-collapse: collapse; font-size: 13px;">
            <tr>
                <td style="background-color: #2A799C; color: white; padding: 5px; font-weight: bold; width: 30%;">Regiao</td>
                <td style="background-color: #C5DCE7; padding: 5px;">{Regiao}</td>
            </tr>
            <tr>
                <td style="background-color: #2A799C; color: white; padding: 5px; font-weight: bold; width: 30%;">UF</td>
                <td style="background-color: #C5DCE7; padding: 5px;">{UF}</td>
            </tr>
            <tr>
                <td style="background-color: #2A799C; color: white; padding: 5px; font-weight: bold;">Município</td>
                <td style="background-color: #C5DCE7; padding: 5px;">{Municipio}</td>
            </tr>
            <tr>
                <td style="background-color: #2A799C; color: white; padding: 5px; font-weight: bold;">Gravidade</td>
                <td style="background-color: #C5DCE7; padding: 5px; font-weight: bold; color: {'red' if Gravidade == 'Com óbito' else 'orange' if Gravidade == 'Com ferido' else 'yellow'};">{Gravidade}</td>
            </tr>
            <tr>
                <td style="background-color: #2A799C; color: white; padding: 5px; font-weight: bold;">Tipo</td>
                <td style="background-color: #C5DCE7; padding: 5px;">{TipoDoSinistro}</td>
            </tr>
            <tr>
                <td style="background-color: #2A799C; color: white; padding: 5px; font-weight: bold;">Causa</td>
                <td style="background-color: #C5DCE7; padding: 5px;">{Causa}</td>
            </tr>
            <tr>
                <td style="background-color: #2A799C; color: white; padding: 5px; font-weight: bold;">UPS</td>
                <td style="background-color: #C5DCE7; padding: 5px;">{UPS}</td>
            </tr>
        </table>
    </div>
    """
    return html

### Criação do pontos no mapa com as informações

In [ ]:
for i in tqdm(range(len(df_mapa))):
    html = fancy_html(i)
    
    iframe = branca.element.IFrame(html=html, width=350, height=275)
    popup = folium.Popup(iframe, max_width=350)

    lat, lon = df_mapa['Latitude'].iloc[i], df_mapa['Longitude'].iloc[i]
    gravidade = df_mapa['Gravidade'].iloc[i]
    
    if gravidade == 'Sem ferido':
        folium.CircleMarker(
            [lat, lon], radius=4, popup=popup,
            color='yellow', fillColor='yellow', fillOpacity=1, weight=1
        ).add_to(marker_cluster_sem_ferido)
    elif gravidade == 'Com ferido':
        folium.CircleMarker(
            [lat, lon], radius=5, popup=popup,
            color='orange', fillColor='orange', fillOpacity=1, weight=1
        ).add_to(marker_cluster_com_ferido)
    elif gravidade == 'Com óbito':
        folium.CircleMarker(
            [lat, lon], radius=6, popup=popup,
            color='red', fillColor='red', fillOpacity=1, weight=1
        ).add_to(marker_cluster_com_obito)

### Construção da caixa de seleção do mapa

In [ ]:
folium.LayerControl().add_to(m)

### Construção da legenda do mapa

In [ ]:
# Legenda otimizada
legend_html = f'''
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 150px; height: 120px; 
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color:white;
                 ">
     &nbsp;<b>Legenda</b><br>
     &nbsp;<span style="color:yellow; font-size:18px;">●</span>&nbsp;Sem ferido<br>
     &nbsp;<span style="color:orange; font-size:18px;">●</span>&nbsp;Com ferido<br>
     &nbsp;<span style="color:red; font-size:18px;">●</span>&nbsp;Com óbito<br>
     &nbsp;<small>{len(df_mapa)} sinistros</small>
      </div>
     '''
m.get_root().html.add_child(folium.Element(legend_html))

### Exportar o mapa

In [ ]:
# Salvar mapa otimizado
m.save('../assets/mapa_sinistros.html')
# m.save('../assets/sinistros_centro_oeste.html')

In [ ]:
# !jupyter nbconvert notebook.ipynb --to slides 